In [2]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [3]:
def authenticate():
  # keys and tokens from the Twitter Dev Console
  consumer_key = '4KzIFprHyNApHjsXOg1OEfJ3c'
  consumer_secret = 'hpBzI2SicsaNqIluwWeK3nySKiDHvSBXmSamKT1cppcXceh59J'
  access_token = '197883124-T8c2XZT0Aa3WGl5cm4Xoe2LZp36lvn5wayU6dHq9'
  access_token_secret = 'NQiqWjqqPkjPgvC7xgZBHHhWaVMqmMSDM6CDf2YDFnAGC'

  # attempt authentication
  try:
    # create OAuthHandler object
    auth = OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    # create tweepy API object to fetch tweets
    print("Authenticating...")
    api = tweepy.API(auth)
    print("API value:....   ", api)
    return api
  except:
    print("Error: Authentication Failed")

In [4]:
API = authenticate()

Authenticating...
API value:....    <tweepy.api.API object at 0x7ff4214b3970>


In [5]:
def clean_tweet(tweet):
  '''
  Utility function to clean tweet text by removing links, special characters
  using simple regex statements.
  '''
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [6]:
def get_tweet_sentiment(tweet):
  '''
  Utility function to classify sentiment of passed tweet
  using textblob's sentiment method
  '''
  # create TextBlob object of passed tweet text
  analysis = TextBlob(clean_tweet(tweet))
  # set sentiment
  if analysis.sentiment.polarity > 0:
    return 'positive'
  elif analysis.sentiment.polarity == 0:
    return 'neutral'
  else:
    return 'negative'

In [7]:
def get_tweets(query, geocode = '53.5500,2.4333,1mi', count = 10):
  '''
  Main function to fetch tweets and parse them.
  '''
  # empty list to store parsed tweets
  tweets = []

  try:
    # call twitter api to fetch tweets
    fetched_tweets = API.search_tweets(q = query, count = count)

    # parsing tweets one by one
    for tweet in fetched_tweets:
      # empty dictionary to store required params of a tweet
      parsed_tweet = {}

      # saving text of tweet
      parsed_tweet['text'] = tweet.text
      # saving sentiment of tweet
      parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text)

      # appending parsed tweet to tweets list
      if tweet.retweet_count > 0:
        # if tweet has retweets, ensure that it is appended only once
        if parsed_tweet not in tweets:
          tweets.append(parsed_tweet)
      else:
        tweets.append(parsed_tweet)

    # return parsed tweets
    return tweets

  except Exception as e:
    # print error (if any)
    print("Error getting tweets", e)


In [8]:
def sentiment_analysis():

  # calling function to get tweets
  tweets = get_tweets(query = 'Education and the UK Government')
  # print(tweets)
  # picking positive tweets from tweets
  ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
  # percentage of positive tweets
  print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
  # picking negative tweets from tweets
  ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
  # percentage of negative tweets
  print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
  # percentage of neutral tweets
  print("Neutral tweets percentage: {} % \
    ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

  # printing first 5 positive tweets
  print("\n\nPositive tweets:")
  for tweet in ptweets[:10]:
    print(tweet['text'])

  # printing first 5 negative tweets
  print("\n\nNegative tweets:")
  for tweet in ntweets[:10]:
    print(tweet['text'])

In [9]:
sentiment_analysis()

Positive tweets percentage: 50.0 %
Negative tweets percentage: 16.666666666666668 %
Neutral tweets percentage: 33.333333333333336 %     


Positive tweets:
RT @FENews: The SEND Review: Why can’t the Government get it right?: The Government’s SEND and Alternative Provision Green Paper and subseq…
RT @Elizabe62115581: @warren00824623 Yes the whole of my family voted for Brexit still feel the same. I do wish we could vote on Devolution…
A government-funded programme has given Suffolk students the chance to study in Thailand with another successful bi… https://t.co/tXGEg7iLlU


Negative tweets:
@jackiebmsp @HumzaYousaf The Scottish government are still paying the bills left behind by labour yet still outperf… https://t.co/22gzu2ABep
